In [1]:
#importing the necessary libraries
import os
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pinecone
import streamlit as st
import openai
from langchain.vectorstores import Pinecone

c:\ProgramData\Anaconda3\envs\araenv\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
st.write("Openai api key:", st.secrets["OPENAI_API_KEY"])
st.write("Pinecone api key:", st.secrets["PINECONE_API_KEY"])

2024-01-14 11:43:10.143 INFO    streamlit.runtime.secrets: Secrets found in multiple locations: C:\Users\aramide\.streamlit\secrets.toml, c:\Users\aramide\Documents\HarmonyHaven\.streamlit\secrets.toml. When multiple secret.toml files exist, local secrets will take precedence over global secrets.


2024-01-14 11:43:12.950 
  command:

    streamlit run C:\Users\aramide\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [3]:
#loading the documents from the data directory
directory = "data"

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

4

In [6]:
"""splitting the document into smaller chunks to ensure the size of the document is manageable and that no relevant inormation
is missed out"""
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

749


In [7]:
#displaying the page content of the splitted document
print(docs[10].page_content)

Blushing, sweating, or trembling Pounding or racing heart Stomachaches Rigid body posture or speaking with an overly soft voice Difficulty making eye contact or being around people they don’t know Feelings of self-consciousness or fear that people will judge them negatively Phobia-related disorders A phobia is an intense fear of—or aversion to—specific objects or situations. Although it can be realistic to be anxious in some circumstances, the fear people with phobias feel is out of proportion to the actual danger caused by the situation or object.

People with a phobia:

May have an irrational or excessive worry about encountering the feared object or situation Take active steps to avoid the feared object or situation Experience immediate intense anxiety upon encountering the feared object or situation Endure unavoidable objects and situations with intense anxiety There are several types of phobias and phobia-related disorders:


In [8]:
#creating embeddings by converting the splitted chunks of text into a format the the AI model can understand
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [9]:
#checking the dimension of the embedded query
query_result = embeddings.embed_query("Hello world")
len(query_result)

384

In [12]:
#storing the embeddings in vector database pinecone
pinecone.init(      
	api_key=st.secrets["PINECONE_API_KEY"],      
	environment="gcp-starter"      
)      
index_name = "mentalhealth-chatbot"

index = Pinecone.from_documents(docs, embeddings, index_name=index_name) 

In [13]:
pinecone.init(api_key=st.secrets["PINECONE_API_KEY"], environment="gcp-starter")
pinecone.list_indexes()

['mentalhealth-chatbot']

In [14]:
#accessing the embedding by using the similarity search function
def get_similar_docs(query, k=1, score= False):
  if score:
    similar_docs = index.similarity_search_with_score(query, k=k)
  else:
    similar_docs = index.similarity_search(query, k=k)
  return similar_docs

query = "What is depression"
similar_docs = get_similar_docs(query)
similar_docs

[Document(page_content='Collapse All What is depression? Depression is a mental health illness when someone feels sad (including crying often), empty, or hopeless most of the time (or loses interest in or takes no pleasure in daily activities) for at least 2 weeks. Depression affects a person’s ability to work, go to school, or have relationships with friends and family. Depression is one of the most common mental health conditions in the United States.2 It is an illness that involves the body, mood, and thoughts. It can affect the way you eat and sleep, the way you feel about yourself, and the way you think about things.\n\nIt is different from feeling “blue” or “down” or just sad for a few hours or a couple of days. Depression is also different from grief over losing a loved one or experiencing sadness after a trauma or difficult event. It is not a condition that can be willed or wished away. People who have depression cannot just “pull themselves” out of it.', metadata={'source': 'd